# Linear models

In [83]:
import matplotlib
from matplotlib import pyplot as plt
matplotlib.pyplot.style.use('ggplot')

from sklearn import datasets, linear_model, metrics, model_selection, pipeline, preprocessing, multiclass, naive_bayes, svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import pandas as pd
import ast
import string
import re
import pickle 

In [3]:
from tqdm import tqdm

In [4]:
data_q = pd.read_csv('data/lenta-ru-news.csv',sep=',')


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
data_q.date = data_q.date.apply(lambda i: int(i[:4]))
train = data_q[data_q.date > 2000]

In [6]:
train.topic.unique()

array(['Россия', 'Спорт', 'Экономика', 'Мир', 'Из жизни',
       'Интернет и СМИ', 'Культура', 'Силовые структуры',
       'Наука и техника', 'Бывший СССР', nan, 'Дом', 'Сочи', 'ЧМ-2014',
       'Путешествия', 'Библиотека', 'Ценности', 'Легпром', 'Бизнес',
       'МедНовости', 'Оружие', '69-я параллель', 'Культпросвет ', 'Крым'],
      dtype=object)

In [7]:
train.tags

19262                   Все
19263                   Все
19264                   Все
19265                   Все
19266                   Все
                ...        
800970           ТВ и радио
800971                  Все
800972             Политика
800973             Общество
800974    Английский футбол
Name: tags, Length: 781713, dtype: object

In [8]:
topic = {'Прибалтика':'Мир', 
 'Кино':'Культура', 
 'Преступность':'Преступность', 
 'Общество':'Из жизни',
 'Происшествия':'Из жизни', 
 'Искусство':'Культура', 
 'Бизнес':'Экономика', 
 'Техника':'Наука и техника', 
 'ТВ и радио':'Интернет и СМИ',
 'Политика':'Мир', 
 'Пресса':'Интернет и СМИ', 
 'Музыка':'Из жизни', 
 'Люди':'Из жизни', 
 'Звери':'Из жизни', 
 'Игры':'Наука и техника',
 'Госэкономика':'Экономика', 
 'Гаджеты':'Наука и техника', 
 'Наука':'Наука и техника', 
 'Еда':'Из жизни', 
 'Рынки':'Экономика', 
 'Деньги':'Экономика',   
 'Летние виды':'Спорт', 
 'Интернет':'Интернет и СМИ', 
 'Театр':'Культура', 
 'Конфликты':'Преступность', 
 'Реклама':'Из жизни',    
 'Космос':'Наука и техника', 
 'Бокс и ММА':'Спорт', 
 'Футбол':'Спорт', 
 'Книги':'Из жизни', 
 'Зимние виды':'Спорт',    
 'Достижения':'Наука и техника', 
 'Coцсети':'Интернет и СМИ', 
 'Вещи':'Из жизни', 
 'События':'Из жизни', 
 'Средняя Азия':'Мир',   
 'Украина':'Мир', 
 'Закавказье':'Мир', 
 'Белоруссия':'Мир', 
 'Молдавия':'Мир', 
 'Софт':'Наука и техника',     
 'Квартира':'Из жизни', 
 'Город':'Россия', 
 'Дача':'Из жизни', 
 'Офис':'Из жизни', 
 'Оружие':'Преступность', 
 'Мнения':'Из жизни',   
 'Первая мировая':'Мир', 
 'Москва':'Россия', 
 'Регионы':'Россия', 
 'Полиция и спецслужбы':'Преступность',
 'Криминал':'Преступность', 
 'Следствие и суд':'Преступность', 
 'Движение':'Из жизни', 
 'Производители':'Экономика',    
 'Мировой бизнес':'Экономика', 
 'Финансы компаний':'Экономика', 
 'Деловой климат':'Россия', 
 'Мир':'Мир',
 'Россия':'Россия', 
 'Часы':'Из жизни', 
 'Явления':'Наука и техника', 
 'Стиль':'Из жизни', 
 'Инструменты':'Наука и техника', 
 'Вооружение':'Преступность',  
 'Вкусы':'Из жизни', 
 'Страноведение':'Мир', 
 'Госрегулирование':'Россия', 
 'История':'Культура',    
 'Внешний вид':'Из жизни', 
 'Автобизнес':'Экономика', 
 'Аналитика рынка':'Экономика', 
 'Туризм':'Из жизни', 
 'Выборы':'Россия',   
 'Экология':'Мир', 
 'Мемы':'Интернет и СМИ', 
 'Мировой опыт':'Мир', 
 'Инновации':'Экономика', 
 'Хоккей':'Спорт', 
 'Вирусные ролики':'Интернет и СМИ', 
 'Фотография':'Из жизни', 
 'Авто':'Из жизни', 
 'Наследие':'Россия',
 'Преступная Россия':'Преступность', 
 'Жизнь':'Из жизни', 
 'Киберпреступность':'Преступность',
 'Социальная сфера':'Из жизни', 
 'Казахстан':'Мир', 
 '69-я параллель':'Из жизни', 
 'Экономика':'Экономика', 
 'Нацпроекты':'Экономика', 
 'Английский футбол':'Спорт',
 'Россия':'Россия', 
 'Спорт':'Спорт',  
 'Мир':'Мир', 
 'Из жизни':'Из жизни',
 'Интернет и СМИ':'Интернет и СМИ', 
 'Культура':'Культура', 
 'Силовые структуры':'Преступность',    
 'Наука и техника':'Наука и техника', 
 'Бывший СССР':'Мир', 
 'Дом':'Из жизни', 
 'Сочи':'Россия', 
 'ЧМ-2014':'Спорт',     
 'Путешествия':'Мир', 
 'Библиотека':'Культура', 
 'Ценности':'Экономика', 
 'Легпром':'Экономика', 
 'Бизнес':'Экономика',
 'МедНовости':'Экономика', 
 'Оружие':'Преступность', 
 'Культпросвет':'Культура', 
 'Крым':'Россия'}

In [9]:
stop = pd.read_csv('data/stopwords.txt', header=None)
stop

,0
0,c
1,а
2,алло
3,без
4,белый
...,...
554,этому
555,этот
556,эту
557,я


In [10]:
train

,url,title,text,topic,tags,date
19262,https://lenta.ru/news/2001/01/01/fire/,Пожар в жилом доме в центре Москвы ликвидирован,Пожар в Москве в жилом здании на Новинском бул...,Россия,Все,2001
19263,https://lenta.ru/news/2001/01/01/junior/,Хоккеисты молодежной сборной России проиграли ...,Молодежная сборная России по хоккею не получит...,Спорт,Все,2001
19264,https://lenta.ru/news/2001/01/01/newlife/,Российская экономика третьего тысячелетия: нов...,Минимальный размер оплаты труда в России с 1 я...,Экономика,Все,2001
19265,https://lenta.ru/news/2001/01/01/tennis/,Кафельников и Южный выигрывают первые матчи се...,В первую неделю нового года начался новый тенн...,Спорт,Все,2001
19266,https://lenta.ru/news/2001/01/01/spb/,Новогодняя петарда устроила пожар на Дворцовой...,В новогоднюю ночь на Дворцовой площади Санкт-П...,Россия,Все,2001
...,...,...,...,...,...,...
800970,https://lenta.ru/news/2019/12/14/shnur/,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,NaN,ТВ и радио,2019
800971,https://lenta.ru/news/2019/12/14/dolg/,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,NaN,Все,2019
800972,https://lenta.ru/news/2019/12/14/dark_euro/,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,NaN,Политика,2019
800973,https://lenta.ru/news/2019/12/14/meteo/,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,NaN,Общество,2019


In [11]:
data_2 = pd.DataFrame(train['text'].str.replace('[^\w\s]', ''))
data_2['text'] = pd.DataFrame(train['title'].str.replace('[^\w\s]', '').str.lower() + data_2['text'].str.lower())
data_2['tags'] = train['tags']
data_2['topic'] = train['topic']
for i in range(len(data_2['tags'])):
    if data_2.tags.iloc(0)[i] == "Все":
        data_2.tags.iloc(0)[i] = data_2.topic.iloc(0)[i]

data_2

C:\Users\krvet\AppData\Local\Temp/ipykernel_15812/2504897993.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_2 = pd.DataFrame(train['text'].str.replace('[^\w\s]', ''))
C:\Users\krvet\AppData\Local\Temp/ipykernel_15812/2504897993.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data_2['text'] = pd.DataFrame(train['title'].str.replace('[^\w\s]', '').str.lower() + data_2['text'].str.lower())


,text,tags,topic
19262,пожар в жилом доме в центре москвы ликвидирова...,Россия,Россия
19263,хоккеисты молодежной сборной россии проиграли ...,Спорт,Спорт
19264,российская экономика третьего тысячелетия новы...,Экономика,Экономика
19265,кафельников и южный выигрывают первые матчи се...,Спорт,Спорт
19266,новогодняя петарда устроила пожар на дворцовой...,Россия,Россия
...,...,...,...
800970,шнуров раскритиковал гагарину на голосепевец с...,ТВ и радио,NaN
800971,в россии предложили изменить правила взыскания...,NaN,NaN
800972,в россии назвали черную дату для европыиспытан...,Политика,NaN
800973,россиянам пообещали аномально теплую погодув б...,Общество,NaN


In [12]:
data = pd.DataFrame()
data['text'] = data_2['text']
data['tags'] = data_2['tags']
data = data.dropna()
data['tags'] = data.tags.apply(lambda i: topic[i])
genres = data.tags.unique()
data['tags'] = data.tags.apply(lambda i: [i])
data

,text,tags
19262,пожар в жилом доме в центре москвы ликвидирова...,[Россия]
19263,хоккеисты молодежной сборной россии проиграли ...,[Спорт]
19264,российская экономика третьего тысячелетия новы...,[Экономика]
19265,кафельников и южный выигрывают первые матчи се...,[Спорт]
19266,новогодняя петарда устроила пожар на дворцовой...,[Россия]
...,...,...
800968,белоруссия одолжит у китая 500 миллионов долла...,[Экономика]
800970,шнуров раскритиковал гагарину на голосепевец с...,[Интернет и СМИ]
800972,в россии назвали черную дату для европыиспытан...,[Мир]
800973,россиянам пообещали аномально теплую погодув б...,[Из жизни]


In [44]:
data.iloc[]

text    в петербурге прошли похороны 11 моряков с курс...
tags                                             [Россия]
Name: 38526, dtype: object

In [13]:
genres

array(['Россия', 'Спорт', 'Экономика', 'Мир', 'Из жизни',
       'Интернет и СМИ', 'Культура', 'Наука и техника', 'Преступность'],
      dtype=object)

# Классификация

In [14]:
train_data, test_data, Y, Y_test = model_selection.train_test_split(data.text, data.tags, test_size = 0.1, random_state = 42)

In [15]:
lb = preprocessing.MultiLabelBinarizer(classes = genres)
train_ans = pd.DataFrame(lb.fit_transform(Y), columns = genres)
test_ans = pd.DataFrame(lb.fit_transform(Y_test), columns = genres)

In [62]:
test_data

716116    в результате крушения катамарана на волге поги...
353423    суд разрешил арестовать контролера из управлен...
254135    в европе началась неделя толерантностив воскре...
443592    на аукционе продали котелок и трость чарли чап...
755736    появилась версия волны терактов в шриланкемасш...
                                ...                        
675466    сотрудница мособлсуда рассказала об устроенной...
537620    альтернативные земным формы жизни допустили на...
414122    оак заподозрили в попытке уволить тысячу сотру...
568684    американские военные попытались ликвидировать ...
606359    названы самые интеллигентные районы центра мос...
Name: text, Length: 75432, dtype: object

**Modeling**

In [17]:
model = multiclass.OneVsRestClassifier(estimator= naive_bayes.BernoulliNB())

In [18]:
tfidf_pipe = pipeline.Pipeline(steps = [('vectorizer', TfidfVectorizer(min_df = 2, max_df = 1000)), 
                                        ('classifier', model)])

In [19]:
tfidf_pipe.fit(train_data, train_ans)

Pipeline(steps=[('vectorizer', TfidfVectorizer(max_df=1000, min_df=2)),
                ('classifier', OneVsRestClassifier(estimator=BernoulliNB()))])

In [20]:
predict = tfidf_pipe.predict(test_data)

In [23]:
qwer = pd.DataFrame(predict, columns = genres)
metrics.f1_score(test_ans, qwer, average = 'samples')

0.7289214126630608

In [86]:
a = data.iloc[3222]
x = pd.DataFrame()
a

text    вкус других объявлен лучшим французским фильмо...
tags                                           [Культура]
Name: 22484, dtype: object

In [87]:
x['text'] = [a.text.replace('[^\w\s]', '').lower()]

In [88]:
x

,text
0,вкус других объявлен лучшим французским фильмо...


In [93]:
qw = tfidf_pipe.predict([a.text.replace('[^\w\s]', '').lower()])
qw

array([[0, 0, 0, 0, 0, 0, 1, 0, 0]])

In [92]:
pd.DataFrame(qw, columns = genres).max()

Россия             0
Спорт              0
Экономика          0
Мир                0
Из жизни           0
Интернет и СМИ     0
Культура           1
Наука и техника    0
Преступность       0
dtype: int32

In [85]:
pickle.dump(tfidf_pipe, open('model.save', 'wb'))

In [99]:
class Classificator(Statistics):
    def __init__(self):
        self.builder = pickle.load(open('model.save', 'rb'))
        self.name = 'Classificator'
        self.ganers = ['Россия', 'Спорт', 'Экономика', 'Мир', 'Из жизни','Интернет и СМИ', 'Культура', 
                       'Наука и техника', 'Преступность']

    def predict(self, news: List[News], *args, **kwargs) -> StatisticsModels:
        news_texts = [one_news.content for one_news in news if one_news.content]
        news_texts = self._cut_non_cyrillic_characters(news_texts)
        if not news_texts:
            return StatisticsModels(type=self.name, stats='none')
        ans_list = self.builder.predict(news_texts)
        ans_list = [self.ganers[np.where(ans == 1)[0][0]] for ans in ans_list]
        
        return StatisticsModels(type=self.name, stats=ans_list)

    @staticmethod
    def _cut_non_cyrillic_characters(news_texts: List[str]) -> List[str]:
        news_texts = [text.replace('[^\w\s]', '').lower() for text in news_texts]
        return news_texts

NameError: name 'List' is not defined

In [116]:
class Classificator():
    def __init__(self):
        self.builder = pickle.load(open('model.save', 'rb'))
        self.name = 'Classificator'
        self.ganers = ['Россия', 'Спорт', 'Экономика', 'Мир', 'Из жизни','Интернет и СМИ', 'Культура', 
                       'Наука и техника', 'Преступность']

    def predict(self, news):
        news_texts = [one_news.text for one_news in news]
        news_texts = self._cut_non_cyrillic_characters(news_texts)
        ans_list = self.builder.predict(news_texts)
        ans_list = [self.ganers[np.where(ans == 1)[0][0]] for ans in ans_list]
        
        return ans_list

    @staticmethod
    def _cut_non_cyrillic_characters(news_texts):
        news_texts = [text.replace('[^\w\s]', '').lower() for text in news_texts]
        return news_texts

In [117]:
qwerty = Classificator()

In [118]:
qwerty.predict([data.iloc[3222],data.iloc[312343],data.iloc[2]])

['Культура', 'Россия', 'Экономика']

text    российская экономика третьего тысячелетия новы...
tags                                          [Экономика]
Name: 19264, dtype: object

In [154]:
q = pd.DataFrame(columns = ['Score', 'F1'])
for k in range(1, 11):
    p = k * 100
    tfidf_pipe = pipeline.Pipeline(steps = [('vectorizer', TfidfVectorizer(min_df = 2, max_df = 1000)), ('classifier', model)])
    tfidf_pipe.fit(train_data, train_ans)
    predict = tfidf_pipe.predict(test_data)
    qwer = pd.DataFrame(predict, columns = genres)
    t = metrics.f1_score(test_ans, qwer, average = 'samples')
    print([p, t])

[100, 0.570239073728972]
[200, 0.6543078849951265]
[300, 0.6863665287941457]
[400, 0.702159119029943]
[500, 0.7115494750238627]
[600, 0.7176030508714251]
[700, 0.7221656591367058]
[800, 0.7247189521688409]
[900, 0.7275749460883091]
[1000, 0.7289214126630608]
